# Model

In [16]:
using Revise, ReachabilityAnalysis, BenchmarkTools, SparseArrays, Plots;

In [2]:
LazySets.set_rtol(Float64, 1e-13)
LazySets.set_ztol(Float64, 1e-13)

# model without parameter variation
function embrake_no_pv(; Tsample=1.E-4, ζ=1e-6, x0=0.05)
    # model's constants
    L = 1.e-3
    KP = 10000.
    KI = 1000.
    R = 0.5
    K = 0.02
    drot = 0.1
    i = 113.1167

    # state variables: [I, x, xe, xc]
    A = Matrix([-(R+K^2/drot)/L 0      KP/L  KI/L;
                          K/i/drot      0      0      0;
                          0             0      0      0;
                          0             0      0      0])

    EMbrake = @system(x' = Ax)

    # initial conditions
    I₀  = Singleton([0.0])
    x₀  = Singleton([0.0])
    xe₀ = Singleton([0.0])
    xc₀ = Singleton([0.0])
    X₀ = I₀ × x₀ × xe₀ × xc₀

    # reset map
    Ar = sparse([1, 2, 3, 4, 4], [1, 2, 2, 2, 4], [1., 1., -1., -Tsample, 1.], 4, 4)
    br = sparsevec([3, 4], [x0, Tsample*x0], 4)
    reset_map(X) = Ar * X + br

    # hybrid system with clocked linear dynamics
    ha = HACLD1(EMbrake, reset_map, Tsample, ζ)
    return IVP(ha, X₀)
end

embrake_no_pv (generic function with 1 method)

In [3]:
# model with parameter variation changing only 1 coefficient, corresponds to the Flow* settings in [SO15]
function embrake_pv_1(; Tsample=1.E-4, ζ=1e-6, Δ=3.0, x0=0.05)
    # model's constants
    L = 1.e-3
    KP = 10000.
    KI = 1000.
    R = 0.5
    K = 0.02
    drot = 0.1
    i = 113.1167
    p = 504. + (-Δ .. Δ)

    # state variables: [I, x, xe, xc]
    A = IntervalMatrix([-p            0      KP/L   KI/L;
                        K/i/drot      0      0      0;
                        0             0      0      0;
                        0             0      0      0])

    EMbrake = @system(x' = Ax)

    # initial conditions
    I₀  = Singleton([0.0])
    x₀  = Singleton([0.0])
    xe₀ = Singleton([0.0])
    xc₀ = Singleton([0.0])
    X₀ = I₀ × x₀ × xe₀ × xc₀

    # reset map
    Ar = sparse([1, 2, 3, 4, 4], [1, 2, 2, 2, 4], [1., 1., -1., -Tsample, 1.], 4, 4)
    br = sparsevec([3, 4], [x0, Tsample*x0], 4)
    reset_map(X) = Ar * X + br

    # hybrid system with clocked linear dynamics
    ha = HACLD1(EMbrake, reset_map, Tsample, ζ)
    return IVP(ha, X₀)
end

embrake_pv_1 (generic function with 1 method)

### Specifications

# Reachability settings

# Results

In [7]:
# definitions for the safety properties

# const 
eₓ = sparsevec([2], [1.], 4);
x0 = 0.05;

## BRKDC01

In [18]:
sol_no_pv_no_jit = nothing
prob_no_pv_no_jit = embrake_no_pv(ζ=0., Tsample=1e-4);

In [17]:
@time sol_no_pv_no_jit = solve(prob_no_pv_no_jit, max_jumps=1001,
                               alg=GLGM06(δ=1e-7, max_order=1, static=true, dim=4, ngens=4));
GC.gc()
 ρ(eₓ, sol_no_pv_no_jit)
ρ(eₓ, sol_no_pv_no_jit) < x0

  0.358709 seconds (261.72 k allocations: 228.737 MiB)
ρ(eₓ, sol_no_pv_no_jit) < x0 = true


true

In [19]:
@time sol_no_pv_no_jit = solve(prob_no_pv_no_jit, max_jumps=1001,
                               alg=GLGM06(δ=1e-8, max_order=1, static=true, dim=4, ngens=4));
GC.gc()
@show ρ(eₓ, sol_no_pv_no_jit) < x0

  1.116409 seconds (241.69 k allocations: 1.688 GiB, 5.19% gc time)
ρ(eₓ, sol_no_pv_no_jit) < x0 = true


true

In [ ]:
# plot(sol_no_pv_no_jit, vars=(1, 2), color=:lightblue)

## BRKDC01 (discrete-time)

In [33]:
@time sol_no_pv_no_jit = solve(prob_no_pv_no_jit, max_jumps=1001,
                               alg=GLGM06(δ=1e-7, max_order=1, static=true, dim=4, ngens=4,
                                   approx_model=NoBloating()))
GC.gc()
@show ρ(eₓ, sol_no_pv_no_jit)
ρ(eₓ, sol_no_pv_no_jit) < x0

  0.085722 seconds (100.34 k allocations: 180.522 MiB, 10.14% gc time)
ρ(eₓ, sol_no_pv_no_jit) = 0.04891413127211572


true

In [34]:
sol_no_pv_no_jit = nothing
GC.gc()

In [35]:
@time sol_no_pv_no_jit = solve(prob_no_pv_no_jit, max_jumps=1001,
                               alg=GLGM06(δ=1e-8, max_order=1, static=true, dim=4, ngens=4,
                                   approx_model=NoBloating()))
GC.gc()
@show ρ(eₓ, sol_no_pv_no_jit)
ρ(eₓ, sol_no_pv_no_jit) < x0

  1.037453 seconds (90.32 k allocations: 1.652 GiB, 31.41% gc time)
ρ(eₓ, sol_no_pv_no_jit) = 0.04891367692966815


true

## BRKNC01

In [51]:
sol_no_pv_with_jit = nothing
GC.gc()

In [47]:
prob_no_pv_with_jit = embrake_no_pv(ζ=[-1e-8, 1e-7], Tsample=1e-4);
@time sol_no_pv_with_jit = solve(prob_no_pv_with_jit, max_jumps=1001,
                                 alg=GLGM06(δ=1e-8, max_order=1, static=true, dim=4, ngens=4));

@show ρ(eₓ, sol_no_pv_with_jit)

  1.019562 seconds (268.72 k allocations: 1.696 GiB, 0.33% gc time)
ρ(eₓ, sol_no_pv_with_jit) = 0.04939380910010891


0.04939380910010891

## BRKNC01 (discrete-time)

In [52]:
prob_no_pv_with_jit = embrake_no_pv(ζ=[-1e-8, 1e-7], Tsample=1e-4);
@time sol_no_pv_with_jit = solve(prob_no_pv_with_jit, max_jumps=1001,
                                 alg=GLGM06(δ=1e-8, max_order=1, static=true, dim=4, ngens=4, approx_model=NoBloating()));

@show ρ(eₓ, sol_no_pv_with_jit)

  0.915464 seconds (179.46 k allocations: 1.664 GiB, 0.35% gc time)
ρ(eₓ, sol_no_pv_with_jit) = 0.04935663367632447


0.04935663367632447

## BRKNP01

In [12]:
sol_no_pv_with_jit = nothing
GC.gc()

In [11]:
prob_pv_1_with_jit = embrake_pv_1(ζ=[-1e-8, 1e-7], Tsample=1e-4);
@time sol_pv_1_with_jit = solve(prob_pv_1_with_jit,
                                alg=ASB07(δ=1e-8, max_order=1, static=true, dim=4, ngens=4), max_jumps=1001);

@show ρ(eₓ, sol_pv_1_with_jit)

 11.422489 seconds (82.67 M allocations: 9.596 GiB, 12.41% gc time)
ρ(eₓ, sol_pv_1_with_jit) = 0.053035688704021304


0.053035688704021304

## BRKNP01 (discrete-time)

In [13]:
sol_no_pv_with_jit = nothing
GC.gc()

In [15]:
prob_pv_1_with_jit = embrake_pv_1(ζ=[-1e-8, 1e-7], Tsample=1e-4);
@time sol_pv_1_with_jit = solve(prob_pv_1_with_jit, max_jumps=1001,
                                alg=ASB07(δ=1e-8, max_order=1, static=true, dim=4, ngens=4, approx_model=NoBloating()));

@show ρ(eₓ, sol_pv_1_with_jit)

 13.619282 seconds (83.42 M allocations: 9.620 GiB, 29.77% gc time)
ρ(eₓ, sol_pv_1_with_jit) = 0.05299530421184598


0.05299530421184598